<a href="https://colab.research.google.com/github/KyoungmiKwon/Bigdata_Training_at_ITwill/blob/main/Team_Project/C03_KM_Mecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mecab Colab 설치하기

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:13 https://developer.download.nvidia.com/compute/c

In [ ]:
%env JAVA_HOME "/user/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/user/lib/jvm/java-8-openjdk-amd64"


In [ ]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

Installing automake (A dependency for mecab-ko)
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 5.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0   482k      0  0:00:02  0:00:02 --:--:--  744k
./configure: line 7378: /usr/bin/file: No such file or directory
In file included from viterbi.cpp:14:0:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::_

#import

In [ ]:
import konlpy
from konlpy.tag import Mecab

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer # 토큰화
import pandas as pd                                       # 파일읽기
import numpy as np                                        #
import matplotlib.pyplot as plt                           # 그래프

#뉴스 기사 데이터 불러오기

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Project/C01_KM_Data_LabelingResult.xlsx',sheet_name='Sheet2',header=0)

In [ ]:
data[:2]

,No,Code,Data,Journal,Title,Text,URL,Sample Y/N,In Charge,-1 / 0 / 1
0,52,5930,2021.04.13 17:50,파이낸셜뉴스,"삼성전자 사내식당 2곳, 신세계·풀무원이 운영한다",삼성전자가 사내식당 2곳에 대해 실시한 외부 급식업체 경쟁입찰에서 '신세계푸드'와 ...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
1,87,5930,2021.04.13 03:11,파이낸셜뉴스,"삼성전자 초청, 美 '반도체 회담'‥백악관 ""즉각적 결정 없을 것""",조 바이든 미국 대통령이 12일(현지시간) '반도체 공급망 강화 온라인 회담'에 참...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,-1


In [ ]:
news = data[:10]
news

,No,Code,Data,Journal,Title,Text,URL,Sample Y/N,In Charge,-1 / 0 / 1
0,52,5930,2021.04.13 17:50,파이낸셜뉴스,삼성전자 사내식당 곳 신세계풀무원이 운영한다,삼성전자가 사내식당 곳에 대해 실시한 외부 급식업체 경쟁입찰에서 신세계푸드와 풀무원...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
1,87,5930,2021.04.13 03:11,파이낸셜뉴스,삼성전자 초청 반도체 회담백악관 즉각적 결정 없을 것,조 바이든 미국 대통령이 일현지시간 반도체 공급망 강화 온라인 회담에 참석해 인사말...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,-1
2,190,5930,2021.04.08 14:55,파이낸셜뉴스,특징주로보스타 삼성전자 미래 신성장 동력 로봇산업 육성,파이낸셜뉴스미래 신성장 동력인 로봇산업에 삼성전자와 전자가 힘을 쏟을 것이란 관측이...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
3,207,5930,2021.04.07 20:41,파이낸셜뉴스,삼성전자 분기역대급 성적표,영업익 전망 웃도는 조억전자 조억 사상 최대 삼성전자와 전자가 코로나 장기화에도 ...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
4,233,5930,2021.04.07 15:04,조선비즈,삼성전자 분기 깜짝 실적에도 주가 하락,삼성전자가 올해 분기 깜짝 실적을 기록했는데도 주가가 하락하고 있다 일 오후 시 분...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,-1
5,243,5930,2021.04.07 09:26,파이낸셜뉴스,삼성운용 내 마음속 원픽 펀드 이벤트 진행,년베스트펀드운용펀드매니저가직접유튜브에출연 파이낸셜뉴스 삼성자산운용은 일 자사 유...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
6,357,5930,2021.04.05 18:33,파이낸셜뉴스,삼성 독주 체제 선택권 축소 우려 단말기 가격 오르나 소비자 부,국내 휴대폰 시장 영향은 전자가 스마트폰 사업에서 철수를 결정하면서 국내 스마트폰...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,-1
7,360,5930,2021.04.05 14:00,파이낸셜뉴스,삼성전자 올해도 기초과학 분야에 억 통 큰 지원,년 상반기 삼성미래기술육성사업 지원과제에 선정된 왼쪽부터서울대학교 류경석 교수 포스...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
8,378,5930,2021.04.05 11:00,파이낸셜뉴스,관리의 삼성 차세대 경영혁신 플랫폼 도입,삼성전자의 새로운 비지니스 플랫폼인 의 로고 파이낸셜뉴스 삼성전자가 차세대 전사자...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,1
9,400,5930,2021.04.05 06:03,조선비즈,주간증시전망 삼성전자와 미국의 성적표를 살펴보자,지난주월일월일 코스피지수는 일 최저치일 최고치 사이에서 오르내렸다 이 기간 국내 증...,https://finance.naver.com/item/news_read.nhn?a...,Y,JY,0


# 텍스트 정제 - 특정 품사 추출, 불용어 제거, 낮은 빈도 단어 제거

In [ ]:
mc = Mecab() # 품사태그 사전

''' # Mecab Method #
Mecab.nouns()  # 명사 추출
Mecab.morphs() # 형태소 추출
Mecab.pos()    # 품사 부착
'''

' # Mecab Method #\nMecab.nouns()  # 명사 추출\nMecab.morphs() # 형태소 추출\nMecab.pos()    # 품사 부착\n'

## 1) 한글 제외한 문자 제거

In [ ]:
# 기사에서 한글 제외하고 모두 제거
news['Text'] = news['Text'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]","")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## 2) 특정 품사만 추출

[Mecab 품사사전](https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=589544265)

In [ ]:
# 1) 기사에서 특정 품사 추출 (기사별)

tag_cls = ['NNG','VV+EC','XSV+EP', 'SL']
# NNG = 일반명사, VV = 동사, EC = 연결어미, XSV = 동사파생접미사, EP = 선어말 어미, SL = 외국어

artc = []
for line in news['Text'][:]:
    temp = []  
    for word in mc.pos(line):
        if word[1] in tag_cls:
            temp.append(word[0])
    artc.append(temp)

print(artc[:])
print(len(artc))

[['사내', '식당', '곳', '대해', '실시', '외부', '급식', '업체', '경쟁', '입찰', '신세계', '푸드', '푸드', '컬처', '운영', '업체', '선정', '됐', '일', '월', '단체', '급식', '다양', '메뉴', '향상', '서비스', '통해', '임직원', '복리', '후생', '향상', '사업장', '기흥', '사업장', '사내', '식당', '곳', '대해', '공개', '입찰', '공고', '했', '업체', '참여', '이번', '입찰', '서류', '심사', '거쳐', '메뉴', '구성', '서비스', '평가', '프레젠테이션', '업체', '인프라', '위생', '점검', '현장', '실사', '임직원', '음식', '품평회', '단계', '공정', '절차', '통해', '진행', '됐', '기존', '단체', '급식', '공급', '했', '스토리', '입찰', '배제', '됐', '최종', '선정', '신세계', '푸드', '푸드', '컬처', '반', '동안', '고용', '승계', '업무', '인수', '후', '월', '일', '운영', '시작', '예정', '관계자', '신규', '업체', '임직원', '만족도', '운영', '보완', '점', '검토', '후', '사내', '식당', '대해서', '경쟁', '입찰', '확대', '계획', '말', '했', '기자', '저작', '뉴스', '무단', '전재', '배포', '금지', '뉴스'], ['대통령', '현지', '시간', '반도체', '공급', '망', '강화', '온라인', '회담', '참석', '인사말', '실시간', '중계', '뉴스', '백악관', '세계', '기업', '불러', '반도체', '공급', '부족', '대해', '기업', '의견', '가운데', '이번', '회담', '결정', '발표', '백악관', '대변인', '현지', '시간', '정례', '브리핑', '대통령', '회의', '참석', '이유', '기업', '의견',

In [ ]:
# 2) 기사에서 특정품사 추출 (모든 기사에서) -- 

tag_cls = ['NNG','VV+EC','XSV+EP', 'SL']
# NNG = 일반명사, VV = 동사, EC = 연결어미, XSV = 동사파생접미사, EP = 선어말 어미, SL = 외국어

voca = []
for line in news['Text'][:]:
    temp = []  
    for word in mc.pos(line):
        if word[1] in tag_cls:
            voca.append(word[0])

print(voca[:])
print(len(voca))

['사내', '식당', '곳', '대해', '실시', '외부', '급식', '업체', '경쟁', '입찰', '신세계', '푸드', '푸드', '컬처', '운영', '업체', '선정', '됐', '일', '월', '단체', '급식', '다양', '메뉴', '향상', '서비스', '통해', '임직원', '복리', '후생', '향상', '사업장', '기흥', '사업장', '사내', '식당', '곳', '대해', '공개', '입찰', '공고', '했', '업체', '참여', '이번', '입찰', '서류', '심사', '거쳐', '메뉴', '구성', '서비스', '평가', '프레젠테이션', '업체', '인프라', '위생', '점검', '현장', '실사', '임직원', '음식', '품평회', '단계', '공정', '절차', '통해', '진행', '됐', '기존', '단체', '급식', '공급', '했', '스토리', '입찰', '배제', '됐', '최종', '선정', '신세계', '푸드', '푸드', '컬처', '반', '동안', '고용', '승계', '업무', '인수', '후', '월', '일', '운영', '시작', '예정', '관계자', '신규', '업체', '임직원', '만족도', '운영', '보완', '점', '검토', '후', '사내', '식당', '대해서', '경쟁', '입찰', '확대', '계획', '말', '했', '기자', '저작', '뉴스', '무단', '전재', '배포', '금지', '뉴스', '대통령', '현지', '시간', '반도체', '공급', '망', '강화', '온라인', '회담', '참석', '인사말', '실시간', '중계', '뉴스', '백악관', '세계', '기업', '불러', '반도체', '공급', '부족', '대해', '기업', '의견', '가운데', '이번', '회담', '결정', '발표', '백악관', '대변인', '현지', '시간', '정례', '브리핑', '대통령', '회의', '참석', '이유', '기업', '의견', '문

## 3) 불용어처리
        불용어 list 생성 -> 불용어 제외 한 단어만 append

In [ ]:
swords = ['삼성전자','삼성','기자','인텔','신세계','풀무원','원','픽','내','마음속','억','통','곳','것','파이낸셜','뉴스','억','명','김','말','간','조선','비즈','조선비즈',
          '저작','무단','전재','배포','금지','월일']

X_ts = [] # 신문 기사별, 단어 저장할 리스트

for word in voca:            # 단어를 사전에서 꺼내서
    if word not in swords:   # 불용어 리스트에 단어가 없다면
        X_ts.append(word)    # X_ts에 추가

In [ ]:
cnt_a = []
for i in X_ts:
    for word in i:
        cnt_a.append(word)
len(cnt_a)

3908

In [ ]:
X_ts[:]

## 4) 낮은 빈도 단어 제거

In [ ]:
# 경우1) 단어의 빈도 수
vocab ={}
for title_voca in X_ts:
    for word in title_voca:
        if word not in vocab:
            vocab[word] = 0
            vocab[word] += 1
print(vocab)

{'사내': 1, '식당': 1, '실시': 1, '외부': 1, '급식': 1, '업체': 1, '경쟁': 1, '입찰': 1, '푸드': 1, '앤': 1, '컬처': 1, '운영': 1, '선정': 1, '일': 1, '월': 1, '단체': 1, '다양': 1, '메뉴': 1, '향상': 1, '서비스': 1, '임직원': 1, '복리': 1, '후생': 1, '수원': 1, '사업장': 1, '기흥': 1, '공개': 1, '공고': 1, '개': 1, '참여': 1, '이번': 1, '서류': 1, '심사': 1, '구성': 1, '등': 1, '평가': 1, '프레젠테이션': 1, '인프라': 1, '위생': 1, '점검': 1, '현장': 1, '실사': 1, '음식': 1, '품평회': 1, '단계': 1, '공정': 1, '절차': 1, '진행': 1, '기존': 1, '공급': 1, '웰': 1, '스토리': 1, '배제': 1, '최종': 1, '달': 1, '반': 1, '동안': 1, '고용': 1, '승계': 1, '업무': 1, '인수': 1, '후': 1, '시작': 1, '예정': 1, '관계자': 1, '신규': 1, '만족도': 1, '보완': 1, '점': 1, '검토': 1, '확대': 1, '계획': 1, '안승현': 1, '조': 1, '바이든': 1, '미국': 1, '대통령': 1, '현지': 1, '시간': 1, '반도체': 1, '망': 1, '강화': 1, '온라인': 1, '회담': 1, '참석': 1, '인사말': 1, '유튜브': 1, '실시간': 1, '중계': 1, '캡처': 1, '백악관': 1, '세계': 1, '기업': 1, '부족': 1, '의견': 1, '가운데': 1, '결정': 1, '발표': 1, '젠': 1, '사키': 1, '대변인': 1, '정례': 1, '브리핑': 1, '그': 1, '회의': 1, '이유': 1, '중': 1, '하나': 1, '문제': 1, '지속': 1, 

In [ ]:
# 경우2) 단어 길이 2 이상인 단어의 빈도 수
vocab ={}

for title_voca in X_ts:
    for word in title_voca:
        if len(word) > 1:
            if word not in vocab:
                vocab[word] = 0
                vocab[word] += 1
print(vocab)

{'사내': 1, '식당': 1, '실시': 1, '외부': 1, '급식': 1, '업체': 1, '경쟁': 1, '입찰': 1, '푸드': 1, '컬처': 1, '운영': 1, '선정': 1, '단체': 1, '다양': 1, '메뉴': 1, '향상': 1, '서비스': 1, '임직원': 1, '복리': 1, '후생': 1, '수원': 1, '사업장': 1, '기흥': 1, '공개': 1, '공고': 1, '참여': 1, '이번': 1, '서류': 1, '심사': 1, '구성': 1, '평가': 1, '프레젠테이션': 1, '인프라': 1, '위생': 1, '점검': 1, '현장': 1, '실사': 1, '음식': 1, '품평회': 1, '단계': 1, '공정': 1, '절차': 1, '진행': 1, '기존': 1, '공급': 1, '스토리': 1, '배제': 1, '최종': 1, '동안': 1, '고용': 1, '승계': 1, '업무': 1, '인수': 1, '시작': 1, '예정': 1, '관계자': 1, '신규': 1, '만족도': 1, '보완': 1, '검토': 1, '확대': 1, '계획': 1, '안승현': 1, '바이든': 1, '미국': 1, '대통령': 1, '현지': 1, '시간': 1, '반도체': 1, '강화': 1, '온라인': 1, '회담': 1, '참석': 1, '인사말': 1, '유튜브': 1, '실시간': 1, '중계': 1, '캡처': 1, '백악관': 1, '세계': 1, '기업': 1, '부족': 1, '의견': 1, '가운데': 1, '결정': 1, '발표': 1, '사키': 1, '대변인': 1, '정례': 1, '브리핑': 1, '회의': 1, '이유': 1, '하나': 1, '문제': 1, '지속': 1, '논의': 1, '일부': 1, '설명': 1, '산업': 1, '협력': 1, '의회': 1, '합의': 1, '향후': 1, '사태': 1, '방지': 1, '국가': 1, '차원': 1, '이날': 1, '제이

In [ ]:
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = False )
print(vocab_sorted)
print(len(vocab_sorted))

[('사내', 1), ('식당', 1), ('실시', 1), ('외부', 1), ('급식', 1), ('업체', 1), ('경쟁', 1), ('입찰', 1), ('푸드', 1), ('컬처', 1), ('운영', 1), ('선정', 1), ('단체', 1), ('다양', 1), ('메뉴', 1), ('향상', 1), ('서비스', 1), ('임직원', 1), ('복리', 1), ('후생', 1), ('수원', 1), ('사업장', 1), ('기흥', 1), ('공개', 1), ('공고', 1), ('참여', 1), ('이번', 1), ('서류', 1), ('심사', 1), ('구성', 1), ('평가', 1), ('프레젠테이션', 1), ('인프라', 1), ('위생', 1), ('점검', 1), ('현장', 1), ('실사', 1), ('음식', 1), ('품평회', 1), ('단계', 1), ('공정', 1), ('절차', 1), ('진행', 1), ('기존', 1), ('공급', 1), ('스토리', 1), ('배제', 1), ('최종', 1), ('동안', 1), ('고용', 1), ('승계', 1), ('업무', 1), ('인수', 1), ('시작', 1), ('예정', 1), ('관계자', 1), ('신규', 1), ('만족도', 1), ('보완', 1), ('검토', 1), ('확대', 1), ('계획', 1), ('안승현', 1), ('바이든', 1), ('미국', 1), ('대통령', 1), ('현지', 1), ('시간', 1), ('반도체', 1), ('강화', 1), ('온라인', 1), ('회담', 1), ('참석', 1), ('인사말', 1), ('유튜브', 1), ('실시간', 1), ('중계', 1), ('캡처', 1), ('백악관', 1), ('세계', 1), ('기업', 1), ('부족', 1), ('의견', 1), ('가운데', 1), ('결정', 1), ('발표', 1), ('사키', 1), ('대변인', 1), ('정례', 1

# Tokenizer

In [ ]:
token = Tokenizer()             # tokenizer 생성
token.fit_on_texts(X_ts)        # 토큰화
# print(token.word_index)       # 단어사전

{'성적표': 1, '사내': 2, '식당': 3, '운영': 4, '초청': 5, '반도체': 6, '회담': 7, '백악관': 8, '결정': 9, '특징': 10, '보스': 11, '미래': 12, '성장': 13, '동력': 14, '로봇산업': 15, '육성': 16, '분기역': 17, '대급': 18, '분기': 19, '실적': 20, '주가': 21, '하락': 22, '운용': 23, '펀드': 24, '이벤트': 25, '진행': 26, '독주': 27, '체제': 28, '선택': 29, '축소': 30, '우려': 31, '단말기': 32, '가격': 33, '소비자': 34, '부': 35, '올해': 36, '기초': 37, '과학': 38, '분야': 39, '지원': 40, '관리': 41, '차세대': 42, '경영': 43, '혁신': 44, '플랫': 45, '폼': 46, '도입': 47, '주간': 48, '증시': 49, '전망': 50, '미국': 51}


In [ ]:
#> Before <#
print(len(token.word_index)) # 단어 사전의 단어 갯수
print(token.word_counts.items()) # 단어 별 등장 횟수

51
odict_items([('사내', 1), ('식당', 1), ('운영', 1), ('초청', 1), ('반도체', 1), ('회담', 1), ('백악관', 1), ('결정', 1), ('특징', 1), ('보스', 1), ('미래', 1), ('성장', 1), ('동력', 1), ('로봇산업', 1), ('육성', 1), ('분기역', 1), ('대급', 1), ('성적표', 2), ('분기', 1), ('실적', 1), ('주가', 1), ('하락', 1), ('운용', 1), ('펀드', 1), ('이벤트', 1), ('진행', 1), ('독주', 1), ('체제', 1), ('선택', 1), ('축소', 1), ('우려', 1), ('단말기', 1), ('가격', 1), ('소비자', 1), ('부', 1), ('올해', 1), ('기초', 1), ('과학', 1), ('분야', 1), ('지원', 1), ('관리', 1), ('차세대', 1), ('경영', 1), ('혁신', 1), ('플랫', 1), ('폼', 1), ('도입', 1), ('주간', 1), ('증시', 1), ('전망', 1), ('미국', 1)])


    빈도수 낮은 단어 제거

In [ ]:
dict_cnt = token.word_counts.items() # 단어 별 등장 횟수 : ('단어', 등장횟수) 형태
for word, cnt in dict_cnt:
    print(word, cnt)

사내 1
식당 1
운영 1
초청 1
반도체 1
회담 1
백악관 1
결정 1
특징 1
보스 1
미래 1
성장 1
동력 1
로봇산업 1
육성 1
분기역 1
대급 1
성적표 2
분기 1
실적 1
주가 1
하락 1
운용 1
펀드 1
이벤트 1
진행 1
독주 1
체제 1
선택 1
축소 1
우려 1
단말기 1
가격 1
소비자 1
부 1
올해 1
기초 1
과학 1
분야 1
지원 1
관리 1
차세대 1
경영 1
혁신 1
플랫 1
폼 1
도입 1
주간 1
증시 1
전망 1
미국 1


In [ ]:
thrs = 3 # 빈도 허용 치
words_cnt = len(token.word_index) # 총 단어 갯수
rare_cnt = 0 # 희귀단어 갯수
word_freq = 0 # 모든 단어 등장 횟수
rere_freq = 0 # 희귀 단어 등장 횟수

vocab ={}
dict_cnt = token.word_counts.items() # 단어 별 등장 횟수 : ('단어', 등장횟수) 형태

for word, cnt in dict_cnt:
   if word not in vocab:
       vocab[word] = 0
    vocab[word] += 1
